In [1]:
import wrds
import pandas_datareader
import datetime as dt
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import MonthEnd
from decimal import Decimal
import scipy
from scipy.optimize import minimize

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
load_data = False
data_folder = '/Users/Xinlin/Desktop/Quantitative Asset Management/Problem Sets/PS2/'
wrds_id = 'yxinlin'

min_year = 1926
max_year = 2021

if load_data:
    conn = wrds.Connection(wrds_username=wrds_id)
    # load CRSP bond returns
    bond_raw = conn.raw_sql("""
                          select kycrspid, mcaldt, tmretnua, tmtotout
                          from crsp.tfz_mth
                          where mcaldt between '01/01/"""+str(min_year)+"""' and '12/31/"""+str(max_year)+"""'
                          """)
    
    # load CRSP risk-free rates
    rf_raw = conn.raw_sql("""
                          select caldt, t90ret, t30ret
                          from crsp.mcti
                          where caldt between '01/01/"""+str(min_year)+"""' and '12/31/"""+str(max_year)+"""'
                          """)
    conn.close()
    
    # save csv
    bond_raw.to_csv('bond_raw.csv')
    rf_raw.to_csv('rf_raw.csv')

else:
    bond_raw = pd.read_csv('bond_raw.csv')
    rf_raw = pd.read_csv('rf_raw.csv')                      

### Question 1: Equal-weighted and Value-weighted Bond Market Return

In [4]:
# data cleaning for bond data
bond_raw['date'] = pd.to_datetime(bond_raw['mcaldt'], format='%Y-%m-%d', errors='ignore')
bond_raw['date'] = pd.DataFrame(bond_raw[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)
bond_raw = bond_raw.sort_values(by=['date', 'kycrspid'])
bond = bond_raw[bond_raw['tmretnua'].notna() & bond_raw['tmtotout'].notna()]

In [5]:
# compute equal-weighted return
bond_ew_ret = bond.groupby('date')['tmretnua'].mean().reset_index().rename(columns={'tmretnua':'bond_ew_ret'})

In [6]:
# compute value-weighted return

# find lagged bond amount
bond['new_date'] = bond['date'] + dt.timedelta(days=20)
bond['new_date'] = pd.DataFrame(bond[['new_date']]) + MonthEnd(0)
lbond = bond[['new_date','kycrspid','tmtotout']].rename(columns={'new_date':'date', 'tmtotout':'lag_amount'})
bond = bond[['date','kycrspid','tmretnua','tmtotout']]
bond = bond.merge(lbond, how='inner', on=['date','kycrspid'])
bond = bond[bond['tmretnua'].notna() | bond['lag_amount'].notna()]
bond = bond.drop_duplicates(subset=['date','kycrspid'])

# find sum of lagged bond amount
bond['sum_lag_amount'] = bond.groupby('date')['lag_amount'].transform('sum')
bond['weight'] = bond['lag_amount']/bond['sum_lag_amount']

bond['vw_ret'] = bond['weight'] * bond['tmretnua']
bond_vw_ret = bond.groupby('date')['vw_ret'].sum().reset_index().rename(columns={'vw_ret':'bond_vw_ret'})

In [7]:
# compute bond market value
price = 1000
bond['lag_mv'] = bond['lag_amount']*price
bond_lag_mv = bond.groupby('date')['lag_mv'].sum().reset_index().rename(columns={'lag_mv':'bond_lag_mv'})

In [8]:
monthly_crsp_bond = bond_lag_mv.merge(bond_ew_ret, how='inner', on=['date'])
monthly_crsp_bond = monthly_crsp_bond.merge(bond_vw_ret, how='inner', on=['date'])
monthly_crsp_bond

,date,bond_lag_mv,bond_ew_ret,bond_vw_ret
0,1926-02-28,8.090000e+05,0.005030,0.006154
1,1926-03-31,8.090000e+05,0.002123,0.003881
2,1926-04-30,8.090000e+05,0.006391,0.007441
3,1926-05-31,8.090000e+05,0.001898,0.001472
4,1926-06-30,8.090000e+05,0.003308,0.003750
...,...,...,...,...
1146,2021-08-31,1.758655e+10,-0.001531,-0.001687
1147,2021-09-30,1.761245e+10,-0.006047,-0.006368
1148,2021-10-31,1.797830e+10,-0.001711,-0.001523
1149,2021-11-30,1.802652e+10,0.004755,0.005304


In [9]:
# save output in a csv
monthly_crsp_bond.to_csv('monthly_crsp_bond.csv', encoding='utf-8')

### Question 2: Excess Value-weighted Returns for Stocks and Bonds

In [10]:
# data cleaning for risk-free rates data
rf_raw['date'] = pd.to_datetime(rf_raw['caldt'], format='%Y-%m-%d', errors='ignore')
rf_raw['date'] = pd.DataFrame(rf_raw[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)
rf_raw = rf_raw[rf_raw['t90ret'].notna() & rf_raw['t30ret'].notna()]
monthly_crsp_riskless = rf_raw[['date','t30ret','t90ret']]

In [11]:
# use PS1 output
monthly_crsp_stock = pd.read_csv('monthly_crsp_stock.csv', index_col=False)
monthly_crsp_stock = monthly_crsp_stock[['date','stock_lag_mv','stock_ew_ret','stock_vw_ret']]
monthly_crsp_stock['date'] = pd.to_datetime(monthly_crsp_stock['date'], format='%Y-%m-%d', errors='ignore')

In [12]:
df = monthly_crsp_stock.merge(monthly_crsp_bond, how='inner', on=['date'])
df = df.merge(monthly_crsp_riskless, how='inner', on=['date'])

In [13]:
# compute excess return for stock & bond
df['stock_excess_vw_ret'] = df['stock_vw_ret'] - df['t30ret']
df['bond_excess_vw_ret'] = df['bond_vw_ret'] - df['t30ret']
monthly_crsp_universe = df[['date','stock_lag_mv','stock_excess_vw_ret','bond_lag_mv','bond_excess_vw_ret','t30ret']]
monthly_crsp_universe

,date,stock_lag_mv,stock_excess_vw_ret,bond_lag_mv,bond_excess_vw_ret,t30ret
0,1926-02-28,2.678975e+07,-0.036105,8.090000e+05,0.003386,0.002768
1,1926-03-31,2.601718e+07,-0.067342,8.090000e+05,0.001103,0.002778
2,1926-04-30,2.404646e+07,0.033845,8.090000e+05,0.004369,0.003072
3,1926-05-31,2.519361e+07,0.011961,8.090000e+05,0.001130,0.000342
4,1926-06-30,2.523775e+07,0.051262,8.090000e+05,0.000291,0.003459
...,...,...,...,...,...,...
1146,2021-08-31,4.589669e+10,0.028765,1.758655e+10,-0.001736,0.000049
1147,2021-09-30,4.727925e+10,-0.043714,1.761245e+10,-0.006401,0.000033
1148,2021-10-31,4.522362e+10,0.066476,1.797830e+10,-0.001560,0.000037
1149,2021-11-30,4.826523e+10,-0.015453,1.802652e+10,0.005260,0.000044


### Question 3: Monthly Unlevered and Levered Risk-Parity Portfolio Returns
(as defined by Asness, Frazzini, and Pedersen (2012))

#### value-weighted portfolio

In [14]:
port = monthly_crsp_universe.copy()

# compute value-weighted portfolio excess return
port['sum_lag_mv'] = port['stock_lag_mv'] + port['bond_lag_mv']
port['s_w'] = port['stock_lag_mv']/port['sum_lag_mv']
port['b_w'] = port['bond_lag_mv']/port['sum_lag_mv']
port['excess_vw_ret'] = port['s_w']*port['stock_excess_vw_ret'] + port['b_w']*port['bond_excess_vw_ret']

#### 60/40 portfolio

In [15]:
# compute 60-40 portfolio excess return
port['s_64w'] = 0.6
port['b_64w'] = 0.4
port['excess_60_40_ret'] = port['s_64w']*port['stock_excess_vw_ret'] + port['b_64w']*port['bond_excess_vw_ret']

#### risk-parity (unlevered & levered) portfolio

In [16]:
# compute inverse volatility of past 36-month & cumulative monthly excess return
S = []
B = []
P = []
for i in range(0+36, 1151):
    temp = port[['date','stock_excess_vw_ret','bond_excess_vw_ret']].iloc[i-36:i]
    s = np.std(temp['stock_excess_vw_ret'])
    b = np.std(temp['bond_excess_vw_ret'])
    S = np.append(S, s)
    B = np.append(B, b)
    
for i in range(0+36, 1151):
    temp1 = port[['date','excess_vw_ret']].iloc[0:i]
    p = np.std(temp1['excess_vw_ret'])
    P = np.append(P, p)

# compute risk parity unlevered portfolio weight
sigma = pd.DataFrame(data={'date':port['date'].iloc[36:1151], 'stock_sigma':S, 'stock_inv_sigma':1/S, 
                           'bond_sigma':B ,'bond_inv_sigma':1/B, 'vwport_sigma':P, 'vwport_inv_sigma':1/P})
sigma['unlevered_k'] = 1 / (sigma['stock_inv_sigma'] + sigma['bond_inv_sigma'])
sigma['s_w'] = sigma['unlevered_k']*sigma['stock_inv_sigma']
sigma['b_w'] = sigma['unlevered_k']*sigma['bond_inv_sigma']

sigma['test_k'] = 1
sigma['test_s_lw'] = sigma['test_k']*sigma['stock_inv_sigma']
sigma['test_b_lw'] = sigma['test_k']*sigma['bond_inv_sigma']

In [17]:
# compute risk-parity unlevered portfolio excess return
rp = monthly_crsp_universe.copy()
rp = rp.merge(sigma, how='inner', on=['date'])
rp['excess_unlevered_rp_ret'] = rp['s_w']*rp['stock_excess_vw_ret'] + rp['b_w']*rp['bond_excess_vw_ret']

# compute risk-parity levered portfolio excess return
rp['test_rf_w'] = rp['test_s_lw'] + rp['test_b_lw'] - 1
rp['test_levered_rp_ret'] = rp['test_s_lw']*rp['stock_excess_vw_ret'] + rp['test_b_lw']*rp['bond_excess_vw_ret']- rp['test_rf_w']*rp['t30ret']
rp['levered_k'] = np.std(port['excess_vw_ret']) / np.std(rp['test_levered_rp_ret'])

rp['s_lw'] = rp['levered_k']*rp['stock_inv_sigma']
rp['b_lw'] = rp['levered_k']*rp['bond_inv_sigma']
rp['rf_w'] = rp['s_lw'] + rp['b_lw'] - 1
rp['excess_levered_rp_ret'] = rp['s_lw']*rp['stock_excess_vw_ret'] + rp['b_lw']*rp['bond_excess_vw_ret']- rp['rf_w']*rp['t30ret']


### Question 4: Summary Statistics

In [18]:
# define a function to compute statistics
def stat(x):
    m = np.mean(x)*12*100
    v = np.std(x)*np.sqrt(12)*100
    t = m / (v/np.sqrt(len(x)/12))
    s = m / v
    sk = x.skew()
    k = x.kurtosis()
    return [m, t, v, s, sk, k]

In [19]:
q4 = pd.DataFrame(columns=['Strategy', 'Excess_Return', 't-stat', 'Volatility', 'Sharpe_Ratio', 'Skewness', 'Kurtosis'])

q4.loc[0, 'Strategy'] = 'CRSP stocks'
q4.loc[0, 'Excess_Return'] = stat(monthly_crsp_universe['stock_excess_vw_ret'].iloc[47:1013])[0]
q4.loc[0, 't-stat'] = stat(monthly_crsp_universe['stock_excess_vw_ret'].iloc[47:1013])[1]
q4.loc[0, 'Volatility'] = stat(monthly_crsp_universe['stock_excess_vw_ret'].iloc[47:1013])[2]
q4.loc[0, 'Sharpe_Ratio'] = stat(monthly_crsp_universe['stock_excess_vw_ret'].iloc[47:1013])[3]
q4.loc[0, 'Skewness'] = stat(monthly_crsp_universe['stock_excess_vw_ret'].iloc[47:1013])[4]
q4.loc[0, 'Kurtosis'] = stat(monthly_crsp_universe['stock_excess_vw_ret'].iloc[47:1013])[5]

q4.loc[1, 'Strategy'] = 'CRSP bonds'
q4.loc[1, 'Excess_Return'] = stat(monthly_crsp_universe['bond_excess_vw_ret'].iloc[47:1013])[0]
q4.loc[1, 't-stat'] = stat(monthly_crsp_universe['bond_excess_vw_ret'].iloc[47:1013])[1]
q4.loc[1, 'Volatility'] = stat(monthly_crsp_universe['bond_excess_vw_ret'].iloc[47:1013])[2]
q4.loc[1, 'Sharpe_Ratio'] = stat(monthly_crsp_universe['bond_excess_vw_ret'].iloc[47:1013])[3]
q4.loc[1, 'Skewness'] = stat(monthly_crsp_universe['bond_excess_vw_ret'].iloc[47:1013])[4]
q4.loc[1, 'Kurtosis'] = stat(monthly_crsp_universe['bond_excess_vw_ret'].iloc[47:1013])[5]

q4.loc[2, 'Strategy'] = 'Value-weighted portfolio'
q4.loc[2, 'Excess_Return'] = stat(port['excess_vw_ret'].iloc[47:1013])[0]
q4.loc[2, 't-stat'] = stat(port['excess_vw_ret'].iloc[47:1013])[1]
q4.loc[2, 'Volatility'] = stat(port['excess_vw_ret'].iloc[47:1013])[2]
q4.loc[2, 'Sharpe_Ratio'] = stat(port['excess_vw_ret'].iloc[47:1013])[3]
q4.loc[2, 'Skewness'] = stat(port['excess_vw_ret'].iloc[47:1013])[4]
q4.loc[2, 'Kurtosis'] = stat(port['excess_vw_ret'].iloc[47:1013])[5]

q4.loc[3, 'Strategy'] = '60/40 portfolio'
q4.loc[3, 'Excess_Return'] = stat(port['excess_60_40_ret'].iloc[47:1013])[0]
q4.loc[3, 't-stat'] = stat(port['excess_60_40_ret'].iloc[47:1013])[1]
q4.loc[3, 'Volatility'] = stat(port['excess_60_40_ret'].iloc[47:1013])[2]
q4.loc[3, 'Sharpe_Ratio'] = stat(port['excess_60_40_ret'].iloc[47:1013])[3]
q4.loc[3, 'Skewness'] = stat(port['excess_60_40_ret'].iloc[47:1013])[4]
q4.loc[3, 'Kurtosis'] = stat(port['excess_60_40_ret'].iloc[47:1013])[5]

q4.loc[4, 'Strategy'] = 'RP Unlevered'
q4.loc[4, 'Excess_Return'] = stat(rp['excess_unlevered_rp_ret'].iloc[11:977])[0]
q4.loc[4, 't-stat'] = stat(rp['excess_unlevered_rp_ret'].iloc[11:977])[1]
q4.loc[4, 'Volatility'] = stat(rp['excess_unlevered_rp_ret'].iloc[11:977])[2]
q4.loc[4, 'Sharpe_Ratio'] = stat(rp['excess_unlevered_rp_ret'].iloc[11:977])[3]
q4.loc[4, 'Skewness'] = stat(rp['excess_unlevered_rp_ret'].iloc[11:977])[4]
q4.loc[4, 'Kurtosis'] = stat(rp['excess_unlevered_rp_ret'].iloc[11:977])[5]

q4.loc[5, 'Strategy'] = 'RP'
q4.loc[5, 'Excess_Return'] = stat(rp['excess_levered_rp_ret'].iloc[11:977])[0]
q4.loc[5, 't-stat'] = stat(rp['excess_levered_rp_ret'].iloc[11:977])[1]
q4.loc[5, 'Volatility'] = stat(rp['excess_levered_rp_ret'].iloc[11:977])[2]
q4.loc[5, 'Sharpe_Ratio'] = stat(rp['excess_levered_rp_ret'].iloc[11:977])[3]
q4.loc[5, 'Skewness'] = stat(rp['excess_levered_rp_ret'].iloc[11:977])[4]
q4.loc[5, 'Kurtosis'] = stat(rp['excess_levered_rp_ret'].iloc[11:977])[5]

q4 = q4.set_index('Strategy')
q4


,Excess_Return,t-stat,Volatility,Sharpe_Ratio,Skewness,Kurtosis
Strategy,,,,,,
CRSP stocks,7.031659,3.33418,18.921984,0.371613,0.289536,7.961087
CRSP bonds,1.605924,4.387325,3.28415,0.488992,-0.009398,4.482219
Value-weighted portfolio,4.192388,2.530656,14.86368,0.282056,0.612927,14.5116
60/40 portfolio,4.861365,3.750918,11.628362,0.418061,0.282959,7.876306
RP Unlevered,2.315598,4.858616,4.276107,0.54152,0.046883,4.488407
RP,-2.852299,-1.738255,14.722433,-0.193738,-0.574598,2.336148
